<span style="font-size:50px"><strong>ML4HC Project 1 - Task 4</strong></span>


In [21]:
import os
import pandas as pd
import numpy as np
from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt

# Define paths
data_dir = "../ml4h_data/p1/"#"data"

pd.set_option('display.max_columns', None)


#  Prompting an LLM to solve a time-series problem - Q4.1

In [117]:
import pandas as pd
import ollama
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, average_precision_score
from ollama import Client

client = Client()

def create_patient_summaries(df):
    summaries = {}
    grouped = df.groupby("RecordID")
    for rid, group in grouped:
        features = []
        for var in group["Variable"].unique():
            vals = group[group["Variable"] == var]["Value"]
            if not vals.empty:
                stats = f"{var}: min={vals.min():.2f}, max={vals.max():.2f}, mean={vals.mean():.2f}"
                features.append(stats)
        text = f"Patient {rid} summary: " + ", ".join(features)
        summaries[rid] = text
    return summaries


def filter_summary_with_llm(summary):
    prompt = (
        f"{summary}\n\n"
        "You are given a synthetic ICU patient summary. Each variable includes min, max, and mean values. "
        "Some values are missing (marked as 'nan'). "
        "Your task is to extract and return the **20 most medically informative values** across all features and statistics. "
        "This means you may choose, for example, 'max HR', 'mean Creatinine', 'min Platelets', etc. "
        "You do not need to keep all three (min/max/mean) for each variable—choose only the most clinically relevant ones. "
        "Ignore any values that are missing (nan). Return exactly 20 values, and only the values, formatted like: 'max HR: 120.0'.  Do not return code but apply the task to the data yourself! Do not return any other text or explaations, just the variables and values"
    )

    response = client.chat(
        model="llama3.2",
        messages=[{
            "role": "user",
            "content": prompt
        }]
    )
    return response["message"]["content"].strip()



In [ ]:
def query_llm(prompt):
    response = client.chat(
        model="llama3.2",
        messages=[{
            "role": "user",
            "content": (
                prompt +
                "\nGiven the above anonymized and synthetic patient summary from a simulated dataset, "
                "use general medical knowledge and statistical reasoning to assess the *likelihood* that this hypothetical patient "
                "might die during their hospital stay. This is purely a research simulation and no real patients are involved. "
                "Please respond with your reasoning step-by-step, and finally rate the risk on a scale from 1 (very low likelihood) to 10 (very high likelihood). "
                "Important: This is not a diagnostic or treatment recommendation. This task is purely a hypothetical thought experiment."
            )
        }]
    )
    return response["message"]["content"].strip()

def extract_risk_number(response_text):
    extract_prompt = (
        f"The following is a simulated reasoning output from a medical LLM:\n\n"
        f"{response_text}\n\n"
        "Based on this, extract and return ONLY the final numeric risk score from 1 to 10, without any explanation or other text."
    )
    response = client.chat(
        model="llama3.2",
        messages=[{
            "role": "user",
            "content": extract_prompt
        }]
    )
    return response["message"]["content"].strip()

def predict_with_llm(summaries, max_patients=None):
    predictions = {}
    for i, (rid, raw_summary) in enumerate(tqdm(summaries.items())):
        if max_patients and i >= max_patients:
            break

        # Filter the summary via LLM
        filtered_summary = filter_summary_with_llm(raw_summary)

        # Main reasoning step
        full_response = query_llm(filtered_summary)

        # Extract score
        score_str = extract_risk_number(full_response)

        try:
            score = float(score_str)
        except ValueError:
            score = None

        predictions[rid] = score
        print(f"\nRecordID: {rid}")
        print("Filtered Summary:\n", filtered_summary)
        print("\nLLM Reasoning:\n", full_response)
        print(f"\n→ Extracted Score: {score}\n")

    return predictions



In [ ]:

def evaluate_predictions(preds, y_true):
    y_true.index = y_true.index.astype(float)  # Ensure keys match type
    y_true = y_true[y_true.index.isin(preds.keys())]

    valid = {float(rid): pred for rid, pred in preds.items() if isinstance(pred, (int, float))}
    y_pred = [valid[rid] for rid in y_true.index if rid in valid]
    y_true_vals = [y_true[rid] for rid in y_true.index if rid in valid]

    print(f"Matched patients: {len(y_pred)}")
    print(f"Example preds: {y_pred[:3]}")
    print(f"Example y_true: {y_true_vals[:3]}")
    print("y_true keys:", set(y_true.index))
    print("Prediction keys:", set(valid.keys()))
    print("Intersected:", set(y_true.index).intersection(valid.keys()))

    if len(y_pred) == 0:
        print("❌ No matched patients — cannot compute AUROC/AUPRC.")
        return

    auroc = roc_auc_score(y_true_vals, y_pred)
    auprc = average_precision_score(y_true_vals, y_pred)
    print(f"LLM AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")


In [ ]:
df = pd.read_parquet("data_c_cleaned.parquet")
df = df.melt(id_vars=["RecordID", "Time"], var_name="Variable", value_name="Value")

# Find RecordIDs where in-hospital death == 1
positive_ids = df[(df["Variable"] == "In-hospital_death") & (df["Value"] == 1)]["RecordID"].unique()

# Filter df to only those records
#df = df[df["RecordID"].isin(positive_ids) & (df["Variable"] != "In-hospital_death")& (df["Variable"] != "ICUType")].reset_index(drop=True)

y_true = df[df["Variable"] == "In-hospital_death"].groupby("RecordID")["Value"].first()
df = df[(df["Variable"] != "In-hospital_death")& (df["Variable"] != "ICUType")].reset_index(drop=True)

 

In [75]:
df

,RecordID,Time,Variable,Value
0,136938.0,0,Age,77.0
1,136938.0,1,Age,77.0
2,136938.0,2,Age,77.0
3,136938.0,3,Age,77.0
4,136938.0,4,Age,77.0
...,...,...,...,...
1058689,138065.0,44,TroponinI,NaN
1058690,138065.0,45,TroponinI,NaN
1058691,138065.0,46,TroponinI,NaN
1058692,138065.0,47,TroponinI,NaN


In [86]:
predssummaries = create_patient_summaries(df)
predssummaries

{132539.0: 'Patient 132539.0 summary: Age: min=54.00, max=54.00, mean=54.00, Gender: min=0.00, max=0.00, mean=0.00, Height: min=nan, max=nan, mean=nan, Weight: min=nan, max=nan, mean=nan, BUN: min=8.00, max=13.00, mean=10.50, Creatinine: min=0.70, max=0.80, mean=0.75, GCS: min=14.00, max=15.00, mean=14.92, Glucose: min=115.00, max=205.00, mean=160.00, HCO3: min=26.00, max=28.00, mean=27.00, HCT: min=30.30, max=33.70, mean=32.50, HR: min=58.00, max=86.00, mean=70.75, K: min=4.00, max=4.40, mean=4.20, Mg: min=1.50, max=1.90, mean=1.70, NIDiasABP: min=39.00, max=67.00, mean=49.70, NIMAP: min=58.67, max=91.00, mean=70.93, NISysABP: min=96.00, max=157.00, mean=113.39, Na: min=136.00, max=137.00, mean=136.50, Platelets: min=185.00, max=221.00, mean=203.00, RespRate: min=12.00, max=24.00, mean=17.39, Temp: min=35.60, max=38.20, mean=37.53, TroponinT: min=nan, max=nan, mean=nan, Urine: min=0.00, max=450.00, mean=144.44, WBC: min=9.40, max=11.20, mean=10.30, ALP: min=nan, max=nan, mean=nan, ALT

In [85]:
predssummaries

"import pandas as pd\n\n# Given data\ndata = {\n    'Patient 132592.0': [\n        {'Age': 'min=35.00, max=35.00, mean=35.00'},\n        {'HR': 'min=100.00, max=120.00, mean=110.00'},\n        {'Creatinine': 'min=1.20, max=2.40, mean=1.80'},\n        {'Platelets': 'min=150.00, max=250.00, mean=200.00'},\n        {'BUN': 'min=10.00, max=30.00, mean=20.00'},\n        {'GFR': 'min=70.00, max=100.00, mean=90.00'}\n    ]\n}\n\n# Initialize an empty list to store the values\nvalues = []\n\n# Iterate over each patient's data\nfor i, patient in enumerate(data['Patient 132592.0']):\n    # Iterate over each feature and statistic\n    for feature in ['Age', 'HR', 'Creatinine', 'Platelets', 'BUN', 'GFR']:\n        if feature in patient:\n            stat = list(patient[feature].split(', '))\n            \n            # Filter out nan values\n            filtered_stat = [x for x in stat if not x.startswith('nan=')]\n            \n            # Add the max and min values to the list\n            if 

In [102]:
preds = predict_with_llm(predssummaries, max_patients=20)
evaluate_predictions(preds, y_true)



  0%|          | 1/4000 [00:47<52:16:55, 47.07s/it]


RecordID: 132539.0
Filtered Summary:
 Here are the 20 most medically informative values extracted from the patient summary:

1. max BUN: 13.00
2. mean Creatinine: 0.75
3. min Platelets: 185.00
4. max NIMAP: 91.00
5. max NISysABP: 157.00
6. max HR: 86.00
7. min HR: 58.00
8. mean Glucose: 160.00
9. max WBC: 11.20
10. mean GCS: 14.92
11. max SysABP: 157.00
12. max Diastolic Blood Pressure (DiasABP): 67.00
13. min Respiratory Rate (RespRate): 12.00
14. max Temp: 38.20
15. mean Temperature: 37.53
16. max Urine Output (Urine): 450.00
17. min Arterial Oxygen Saturation (SaO2): nan
18. max PaCO2: nan
19. mean Blood Pressure (MAP): nan
20. min Creatinine Clearance (not available in the data, using a default value of 30)

LLM Reasoning:
 To assess the likelihood of this patient dying during their hospital stay, I'll evaluate each value in the context of general medical knowledge and statistical reasoning.

1. **max BUN: 13.00**: Elevated BUN can indicate dehydration, kidney disease, or liver di

  0%|          | 2/4000 [01:18<41:51:46, 37.70s/it]


RecordID: 132540.0
Filtered Summary:
 Here are the 20 most medically informative values extracted from the patient summary:

1. max HR: 90.0
2. mean Creatinine: 1.10
3. min Platelets: 135.00
4. mean BUN: 18.33
5. max SysABP: 138.0
6. mean GCS: 13.33
7. min DiasABP: 46.00
8. mean HCT: 28.66
9. max MAP: 98.00
10. mean HR: 78.77
11. min SaO2: 93.00
12. mean NIMAP: 77.22
13. max WBC: 13.30
14. mean K: 3.90
15. min Glucose: 105.00
16. mean FiO2: 0.45
17. max PaCO2: 46.00
18. mean GCS: 13.33
19. max PaO2: 445.00
20. mean NIDiasABP: 58.00

LLM Reasoning:
 I'll provide a step-by-step analysis of the given patient summary to estimate the likelihood of this hypothetical patient dying during their hospital stay.

1. **Vital signs**:
   - Max HR (heart rate): 90.0 beats per minute, which is slightly elevated but not critically high.
   - Mean HCT (hematocrit) and mean NIDiasABP (normalized incident blood pressure) indicate a stable condition, although slightly low.
   - Mean FiO2 (fraction of ins

  0%|          | 3/4000 [01:55<41:46:27, 37.63s/it]

...


RecordID: 132582.0
Filtered Summary:
 Here are the 20 most medically informative values from the patient summary:

1. max HR: 108.0
2. mean Creatinine: 1.10
3. min Platelets: 145.00
4. mean BUN: 30.75
5. max GCS: 15.00
6. min HCT: 25.80
7. mean NIDiasABP: 44.64
8. max K: 4.20
9. mean Glucose: 165.00
10. min HR: 77.00
11. mean WBC: 4.24
12. max SysABP: 138.00
13. mean Temp: 36.40
14. max NIMAP: 81.33
15. min Mg: 1.80
16. mean HCO3: 27.33
17. max DiasABP: 60.00
18. mean RespirRate: 19.58
19. max SaO2 (assuming this is available): not included due to lack of 'SaO2' data, using the next best - mean Platelets: 156.80 
20. min Temp: 35.40

LLM Reasoning:
 To assess the likelihood of this patient dying during their hospital stay, I'll analyze the provided data step by step.

**Step 1: Vital Signs**

The patient's vital signs show:

* High heart rate (max HR: 108.0)
* Elevated mean BUN (30.75), which may indicate kidney dysfunction or dehydration
* Slightly elevated blood glucose (mean Glucos

  0%|          | 19/4000 [11:37<41:50:39, 37.84s/it]


RecordID: 132584.0
Filtered Summary:
 Here are the 20 most medically informative values from the patient summary:

1. max HR: 122.0
2. mean Creatinine: 1.09
3. min Platelets: 49.00
4. mean GCS: 9.92
5. mean BUN: 18.43
6. max DiasABP: 116.0
7. min HCT: 15.10
8. mean TroponinT: 0.19
9. max SysABP: 175.0
10. mean MAP: 86.38
11. min HR: 53.00
12. mean HCO3: 15.71
13. max BUN: 24.00
14. mean GCS: 9.92
15. max NIDiasABP: 129.0
16. mean NIMAP: 88.79
17. min K: 3.50
18. max Platelets: 143.0
19. mean ALP: 49.50
20. max PaO2: 191

LLM Reasoning:
 To assess the likelihood of this patient dying during their hospital stay, I'll analyze each value in the context of general medical knowledge and statistical reasoning.

**Step 1: Evaluate Lab Values**

1. Creatinine (1.09): Elevated, indicating impaired kidney function.
2. TroponinT (0.19): Slightly elevated, suggesting possible myocardial damage or inflammation.
3. BUN (18.43) and max BUN (24.00): Elevated, indicating dehydration, kidney disease, or

  0%|          | 20/4000 [12:50<42:36:57, 38.55s/it]


RecordID: 132585.0
Filtered Summary:
 Here are the 20 most medically informative values:

1. max HR: 101.0
2. mean Creatinine: 0.53
3. min Platelets: 195.0
4. max SysABP: 140.0
5. mean Glucose: 105.0
6. max FiO2: 1.00
7. min BUN: 7.00
8. mean HR: 80.8
9. max TroponinT: nan
10. min WBC: 9.60
11. mean NIDiasABP: 43.5
12. max PaO2: 382.0
13. max Creatinine: 0.6
14. min Na: 136.00
15. mean HCT: 23.94
16. max Mg: 2.20
17. mean GCS: 13.79
18. min DiasABP: 46.0
19. max PaCO2: 70.0
20. mean Temp: 37.18

LLM Reasoning:
 To assess the likelihood of this patient dying during their hospital stay, I'll analyze each value and consider how they relate to overall health and potential complications.

1. **Max HR:** 101.0
   - A high heart rate may indicate stress or underlying cardiovascular conditions.
   - High heart rates are generally associated with increased mortality risk in various clinical settings.

2. **Mean Creatinine:** 0.53
   - This value is relatively low, indicating good kidney functi

In [ ]:
""" might find that performing
few-shot predictions where you previously populate the context with a few examples
of descriptions and predictions from the training set, might give you much better
answers. Be mindful with your context space to keep the model running fast! Report
the test set performance of your predictions. Can you still use AuROC/AuPRC if
getting binary responses? Maybe you can ask the model for a score in a range of
1-10? (2 pts)"""

In [143]:
def query_llm(prompt, few_shot=False, few_shot_examples=None):


    base_prompt = (
        "\n\n This is a purely synthetic and fictional scenario: "
                "These patient records are randomly generated and part of a simulation exercise for research on language model reasoning. "
                "No real patients are involved. You are NOT providing medical advice. "
                "Your task is to evaluate the *simulated* risk of in-hospital death for the last patient based on hypothetical vital signs, using general medical knowledge and reasoning, "
                "and to provide a number from 1 (low risk) to 10 (high risk). Please respond with your reasoning step-by-step, and finally rate the risk on a scale from 1 (very low likelihood) to 10 (very high likelihood), ONLY for the last patient. "
                "Important: This is not a diagnostic or treatment recommendation. This task is purely a hypothetical thought experiment."
    )

    messages = []

    if few_shot and few_shot_examples:
        label_map = {0: "In no significant danger", 1: "At very severe risk"}

        messages = [{
            "role": "user",
            "content": base_prompt + "First you will get several artificial example scenarios to guide your response. However the predictons shuld only me made for the last patient."
        }]

        for summary, binary_label in few_shot_examples:
            messages.append({
                "role": "user",
                "content": "Patient Example:\n" + summary
            })
            messages.append({
                "role": "user",
                "content": f"(example) Based on this summary, the patient would be classified as: {label_map[binary_label]}"
            })

    messages.append({
        "role": "user",
        "content": prompt 
    })
    messages.append({
        "role": "assistant",
        "content": base_prompt
    })
    print(messages)

    response = client.chat(
        model="llama3.2",
        messages=messages
    )
    return response["message"]["content"].strip()
    print()
    return response["message"]["content"].strip()


def predict_with_llm(summaries, max_patients=None, few_shot=False, few_shot_examples=None):
    predictions = {}
    for i, (rid, raw_summary) in enumerate(tqdm(summaries.items())):
        if max_patients and i >= max_patients:
            break

        filtered_summary = filter_summary_with_llm(raw_summary)
        full_response = query_llm(filtered_summary, few_shot=few_shot, few_shot_examples=few_shot_examples)
        score_str = extract_risk_number(full_response)

        try:
            score = float(score_str)
        except ValueError:
            score = None

        predictions[rid] = score
        print(f"\nRecordID: {rid}")
        print("Filtered Summary:\n", filtered_summary)
        print("\nLLM Reasoning:\n", full_response)
        print(f"\n→ Extracted Score: {score}\n")

    return predictions


In [129]:
def generate_few_shot_examples_filtered(df, y_true, n=3):
    # 1. Get RecordIDs of n alive and n dead patients
    alive_ids = y_true[y_true == 0].index[:n]
    dead_ids = y_true[y_true == 1].index[:n]
    selected_ids = list(alive_ids) + list(dead_ids)

    # 2. Filter df down to just those patients
    df_small = df[df["RecordID"].isin(selected_ids)].reset_index(drop=True)

    # 3. Generate filtered summaries for selected patients
    all_summaries = create_patient_summaries(df_small)
    few_shot = []

    for rid in selected_ids:
        raw_summary = all_summaries[rid]
        filtered_summary = filter_summary_with_llm(raw_summary)
        label = y_true[rid]
        few_shot.append((filtered_summary, label))

    return few_shot


In [130]:
df = pd.read_parquet("data_a_cleaned.parquet")
df = df.melt(id_vars=["RecordID", "Time"], var_name="Variable", value_name="Value")

y_true = df[df["Variable"] == "In-hospital_death"].groupby("RecordID")["Value"].first()
df = df[(df["Variable"] != "In-hospital_death")& (df["Variable"] != "ICUType")].reset_index(drop=True)
 
few_shot_examples = generate_few_shot_examples_filtered(df, y_true, n=1)
few_shot_examples

[("'min Creatinine: 0.70\n'max BUN: 13.00\n'mean TroponinT: nan\n'min WBC: 9.40\n'max Glucose: 205.00\n'max NIMAP: 91.00\n'max HCO3: 28.00\n'min HR: 58.00\n'mean GCS: 14.92\n'mean NIDiasABP: 49.70\n'max Mg: 1.90\n'max HCT: 33.70\n'max SaO2: nan\n'max Platelets: 221.00\n'min BUN: 8.00\n'mean ALP: nan\n'mean SysABP: nan\n'max Urine: 450.00\n'max MAP: nan\n'mean HCT: 32.50",
  np.float64(0.0)),
 ("'max GCS: 15.00\n'min BUN: 58.00\n'mean Creatinine: 0.68\n'min HCT: 32.60\n'max SysABP: 172.00\n'mean HR: 71.95\n'max TroponinT: nan\n'mean NIDiasABP: 28.72\n'min K: 3.90\n'max HCO3: 18.00\n'mean ALP: 47.00\n'mean PaO2: 130.40\n'min GCS: 8.00\n'max Mg: 4.10\n'mean Urine: 43.81\n'min WBC: 16.10\n'nan FiO2: nan\n'mean DiasABP: 40.31\n'max NIMAP: 92.67",
  np.float64(1.0))]

In [145]:
df = pd.read_parquet("data_c_cleaned.parquet")
df = df.melt(id_vars=["RecordID", "Time"], var_name="Variable", value_name="Value")

# Find RecordIDs where in-hospital death == 1
positive_ids = df[(df["Variable"] == "In-hospital_death") & (df["Value"] == 1)]["RecordID"].unique()

# Filter df to only those records
#df = df[df["RecordID"].isin(positive_ids) & (df["Variable"] != "In-hospital_death")& (df["Variable"] != "ICUType")].reset_index(drop=True)

y_true = df[df["Variable"] == "In-hospital_death"].groupby("RecordID")["Value"].first()
df = df[(df["Variable"] != "In-hospital_death")& (df["Variable"] != "ICUType")].reset_index(drop=True)
predssummaries = create_patient_summaries(df)
predssummaries
 

{152871.0: 'Patient 152871.0 summary: Age: min=71.00, max=71.00, mean=71.00, Gender: min=1.00, max=1.00, mean=1.00, Height: min=167.60, max=167.60, mean=167.60, Weight: min=79.20, max=89.60, mean=86.09, BUN: min=36.00, max=41.00, mean=37.60, Creatinine: min=3.50, max=4.20, mean=3.82, DiasABP: min=44.00, max=88.00, mean=56.55, FiO2: min=0.40, max=0.50, mean=0.44, GCS: min=9.00, max=11.00, mean=10.00, Glucose: min=82.00, max=88.00, mean=83.80, HCO3: min=19.00, max=20.00, mean=19.60, HCT: min=28.60, max=30.40, mean=29.35, HR: min=57.00, max=76.00, mean=62.32, K: min=3.90, max=5.00, mean=4.27, Lactate: min=0.70, max=1.50, mean=1.10, MAP: min=63.00, max=112.00, mean=82.81, Mg: min=1.50, max=2.20, mean=1.93, NIDiasABP: min=nan, max=nan, mean=nan, NIMAP: min=nan, max=nan, mean=nan, NISysABP: min=nan, max=nan, mean=nan, Na: min=138.00, max=142.00, mean=139.80, PaCO2: min=31.00, max=42.00, mean=37.00, PaO2: min=75.00, max=114.00, mean=97.89, Platelets: min=101.00, max=117.00, mean=109.67, RespR

In [ ]:
preds = predict_with_llm(predssummaries, max_patients=10,few_shot=True, few_shot_examples=few_shot_examples)
evaluate_predictions(preds, y_true)



  0%|          | 0/4000 [00:00<?, ?it/s]

[{'role': 'user', 'content': '\n\n This is a purely synthetic and fictional scenario: These patient records are randomly generated and part of a simulation exercise for research on language model reasoning. No real patients are involved. You are NOT providing medical advice. Your task is to evaluate the *simulated* risk of in-hospital death for the last patient based on hypothetical vital signs, using general medical knowledge and reasoning, and to provide a number from 1 (low risk) to 10 (high risk). Please respond with your reasoning step-by-step, and finally rate the risk on a scale from 1 (very low likelihood) to 10 (very high likelihood), ONLY for the last patient. Important: This is not a diagnostic or treatment recommendation. This task is purely a hypothetical thought experiment.First you will get several artificial example scenarios to guide your response. However the predictons shuld only me made for the last patient.'}, {'role': 'user', 'content': "Patient Example:\n'min Cre

#  Using LLMs to retrieve embeddings - Q4.2

In [ ]:
def get_llm_embedding(summary):
    response = client.embeddings(
        model="llama3.2",
        prompt=summary
    )
    return response["embedding"]

def extract_llm_embeddings(summaries, max_patients=None):
    embeddings, record_ids = [], []

    for i, (rid, summary) in enumerate(tqdm(summaries.items())):
        if max_patients and i >= max_patients:
            break
        try:
            emb = get_llm_embedding(summary)
            embeddings.append(emb)
            record_ids.append(rid)
        except Exception as e:
            print(f"Error for {rid}: {e}")
    
    return np.array(embeddings), np.array(record_ids)


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, average_precision_score

def train_probe(X, y, X_test, y_test):
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X, y)
    y_pred = clf.predict_proba(X_test)[:, 1]
    auroc = roc_auc_score(y_test, y_pred)
    auprc = average_precision_score(y_test, y_pred)
    print(f"LLM Embeddings - AUROC: {auroc:.4f}, AUPRC: {auprc:.4f}")


In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

def plot_embeddings_tsne(embeddings, labels):
    reduced = TSNE(n_components=2, perplexity=30).fit_transform(embeddings)
    plt.figure(figsize=(6, 5))
    plt.scatter(reduced[:, 0], reduced[:, 1], c=labels, cmap='coolwarm', alpha=0.7)
    plt.title("LLM Embedding Space (t-SNE)")
    plt.colorbar(label="In-Hospital Death")
    plt.show()


#  Using time-series foundation models - Q4.3